In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
import sklearn.preprocessing
import sklearn.cross_validation



%matplotlib inline

In [2]:
train = pd.read_csv('./train.csv').set_index("ID")
test = pd.read_csv('./test.csv').set_index("ID")
print train.shape
print test.shape

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(145231, 1933)
(145232, 1932)


In [3]:
m,n = train.shape #draw samples from data to determine the type of data

random.seed(1)
r = np.random.choice(m, size = 20000, replace=False)

In [4]:
drop_cols = [c for c in train.columns if len(np.unique(train[c])) == 1]#Should later drop columns from train and test
print drop_cols
train.drop(drop_cols, axis = 1, inplace = True)
test.drop(drop_cols, axis = 1, inplace = True)

['VAR_0847', 'VAR_1428']


/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [5]:
object_cols = train.columns[train.dtypes == 'object']

In [6]:
for c in object_cols:
    print c, Counter(train.loc[r,c]).most_common(20)

VAR_0001 [(nan, 9940), ('R', 5906), ('H', 4120), ('Q', 34)]
VAR_0005 [(nan, 9940), ('B', 4992), ('C', 3734), ('N', 1133), ('S', 201)]
VAR_0008 [(False, 10057), (nan, 9943)]
VAR_0009 [(False, 10057), (nan, 9943)]
VAR_0010 [(False, 10057), (nan, 9943)]
VAR_0011 [(False, 10057), (nan, 9943)]
VAR_0012 [(False, 10057), (nan, 9943)]
VAR_0043 [(False, 10057), (nan, 9943)]
VAR_0044 [('[]', 10057), (nan, 9943)]
VAR_0073 [(nan, 16914), ('13MAR09:00:00:00', 17), ('18MAY12:00:00:00', 15), ('13JUL12:00:00:00', 14), ('07SEP12:00:00:00', 14), ('23APR12:00:00:00', 14), ('25OCT11:00:00:00', 13), ('21DEC11:00:00:00', 13), ('06DEC11:00:00:00', 13), ('16AUG12:00:00:00', 13), ('07NOV11:00:00:00', 13), ('10JUL12:00:00:00', 12), ('02APR12:00:00:00', 12), ('19OCT11:00:00:00', 12), ('05OCT11:00:00:00', 12), ('01JUN12:00:00:00', 12), ('05SEP12:00:00:00', 11), ('10JUN11:00:00:00', 11), ('23JUL12:00:00:00', 11), ('21MAY12:00:00:00', 11)]
VAR_0075 [(nan, 9943), ('22SEP10:00:00:00', 76), ('23SEP10:00:00:00', 69), (

In [7]:
dates_cols =['VAR_0073', 'VAR_0075', 'VAR_0156', 'VAR_0157', 'VAR_0158', 'VAR_0159', 'VAR_0166', 'VAR_0167', 'VAR_0168', 'VAR_0169', 'VAR_0176', 'VAR_0177', 'VAR_0178','VAR_0179', 'VAR_0204', 'VAR_0217']

In [8]:
for c in object_cols:
    if c not in dates_cols:
        print c, Counter(train.loc[r,c]).most_common(20)

VAR_0001 [(nan, 9940), ('R', 5906), ('H', 4120), ('Q', 34)]
VAR_0005 [(nan, 9940), ('B', 4992), ('C', 3734), ('N', 1133), ('S', 201)]
VAR_0008 [(False, 10057), (nan, 9943)]
VAR_0009 [(False, 10057), (nan, 9943)]
VAR_0010 [(False, 10057), (nan, 9943)]
VAR_0011 [(False, 10057), (nan, 9943)]
VAR_0012 [(False, 10057), (nan, 9943)]
VAR_0043 [(False, 10057), (nan, 9943)]
VAR_0044 [('[]', 10057), (nan, 9943)]
VAR_0196 [(False, 10057), (nan, 9943)]
VAR_0200 [(nan, 9943), ('CHICAGO', 138), ('JACKSONVILLE', 83), ('HOUSTON', 82), ('PHILADELPHIA', 72), ('SAN ANTONIO', 66), ('LOS ANGELES', 59), ('INDIANAPOLIS', 51), ('LOUISVILLE', 44), ('COLUMBIA', 44), ('RICHMOND', 43), ('SACRAMENTO', 42), ('LEXINGTON', 40), ('COLUMBUS', 39), ('EL PASO', 39), ('MILWAUKEE', 37), ('DALLAS', 36), ('ATLANTA', 35), ('GREENVILLE', 35), ('FAYETTEVILLE', 34)]
VAR_0202 [('BatchInquiry', 10057), (nan, 9943)]
VAR_0214 [(nan, 10472), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (na

In [9]:
redundant_0008 = ['VAR_0008', 'VAR_0009', 'VAR_0010','VAR_0011','VAR_0012', 'VAR_0043','VAR_0044','VAR_0196', 'VAR_0229', ] #Contains redundant information
train.drop(redundant_0008, axis = 1, inplace = True)
test.drop(redundant_0008, axis = 1, inplace = True)

In [10]:
#Does not work on ubuntu machine
#train['NaNs'] = train.apply(lambda x: np.sum(pd.isnull(x)), axis = 1) #Count the number of NaNs in each row will allow us to separate the types of data
#test['NaNs'] = test.apply(lambda x: np.sum(pd.isnull(x)), axis = 1)

In [ ]:
def to_date(date_str):
    if pd.notnull(date_str):
        return datetime.datetime.strptime(date_str, '%d%b%y:%H:%M:%S')
    else:
        return date_str

In [ ]:
for c in dates_cols:
    train[c] = train[c].apply(to_date)
    test[c] = test[c].apply(to_date)

In [ ]:
diff = list(set(test.columns[test.dtypes == 'datetime64[ns]']) - set(train.columns[train.dtypes == 'datetime64[ns]'])) ##test got converted into date time format while train is still object format

In [ ]:
train.drop(diff, axis = 1, inplace = True)
test.drop(diff, axis = 1, inplace = True)

In [ ]:
dates_cols_fixed = train.columns[train.dtypes == 'datetime64[ns]']

In [ ]:
errors_dates = [] #Does not matter because will convert to float
for c in dates_cols_fixed: 
    try:
        train[c] = train[c].apply(lambda x: x.year)
        test[c] = test[c].apply(lambda x: x.year)
    except:
        errors.append(c)

In [ ]:
object_cols = train.columns[train.dtypes == 'object']

In [ ]:
enc= {}
errors = []
for c in object_cols:
    try:
        enc[c] = LabelEncoder()
        train[c] = enc[c].fit_transform(train[c])
        test[c] = enc[c].transform(test[c])
    except:
        errors.append(c)

In [ ]:
#Will need to adjust errors.. can fit transform on test and train data
print errors # ['VAR_0157', 'VAR_0158', 'VAR_0167', 'VAR_0177', 'VAR_0200', 'VAR_0214', 'VAR_0237', 'VAR_0283', 'VAR_0404', 'VAR_0493']

In [ ]:
train.drop(errors, axis = 1, inplace = True)
test.drop(errors, axis = 1, inplace = True)

In [ ]:
print train.info()
print test.info()

In [ ]:
X = train.drop('target', axis = 1).values
y = train.target.values

In [ ]:
imputer = sklearn.preprocessing.Imputer(strategy='median')
X = imputer.fit_transform(X)

In [ ]:
rf = RandomForestClassifier(n_estimators = 45)

In [ ]:
sklearn.cross_validation.cross_val_score(rf, X, y, scoring = 'roc_auc')

In [ ]:
#ada = AdaBoostClassifier()

In [ ]:
#sklearn.cross_validation.cross_val_score(ada, X, y, scoring = 'roc_auc')

In [ ]:
rf.fit(X,y)

In [ ]:
X_test = test.values
X_test = imputer.transform(X_test)


In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
# MAKING SUBMISSION
submission = pd.DataFrame(y_pred, index=test.index, columns=['target'])
submission.index.name = 'ID'
submission.to_csv('beat_withrf_B.csv')